# Crawling Data

## Twitter

In [ ]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
def tweet_scrape(tag, count=100):
  list_tweet = []
  for i,tweet in enumerate(sntwitter.TwitterHashtagScraper(tag).get_items()):
    if i > count:
      break
    list_tweet.append({
      'Nama': tweet.user.username, 
      'Content': tweet.content, 
      'Lokasi': tweet.user.location
    })
  return pd.DataFrame(list_tweet) 

In [ ]:
tweet = tweet_scrape('capres2024 AND ganjarpranowo')
tweet.to_csv('ganjarScrape.csv', index=False)

<ipython-input-22-cbc57a5e4014>:8: FutureWarning: content is deprecated, use rawContent instead
  'Content': tweet.content,


In [ ]:
!pip3 uninstall twint && pip3 install twint && pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

Found existing installation: twint 2.1.21
Uninstalling twint-2.1.21:
  Would remove:
    /root/.local/bin/twint
    /root/.local/lib/python3.8/site-packages/twint-2.1.21.dist-info/*
    /root/.local/lib/python3.8/site-packages/twint/*
Proceed (Y/n)? Y
  Successfully uninstalled twint-2.1.21
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to /tmp/pip-install-bn8qifv1/twint_15a6b086a4304533a1fde536f4265b64
  Running command git clone --filter=blob:none --quiet https://github.com/twintproject/twint.git /tmp/pip-install-bn8qifv1/twint_15a6b086a4304533a1fde536f4265b64
  Running command git checkout -q origin/master
  Resolved https://github.com/twintproject/twint.git to commit origin/master
  Preparing metadata (setup.py) ... done
  Created wheel for twint: filenam

In [ ]:
!pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import twint
import nest_asyncio
nest_asyncio.apply()

In [ ]:
c = twint.Config()
c.Search = '#capres2024 AND #ganjarpranowo'
c.Lang = 'id'
c.Limit = 1000
# c.Since = "2019-01-01"
# c.Until = "2022-01-01"
c.Pandas = True
twint.run.Search(c)

In [ ]:
Tweets_df = twint.storage.panda.Tweets_df

In [ ]:
Tweets_df.columns

In [ ]:
df = pd.DataFrame({
    'name': Tweets_df['name'],
    'tweet': Tweets_df['tweet'],
    'place': Tweets_df['place'],
})

In [ ]:
df.to_csv('ganjarCapres.csv')

## Web

In [ ]:
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests as req

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import pandas as pd

### pta.trunojoyo.ac.id

In [ ]:
class ScrapingPtaTrunojoyo:
  def __init__(self, url):
    self.url = url
  
  def page(self):
    res = req.get(self.url)
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find('ol', class_='pagination').findChildren(recursive=False)
    return int(contents[-3].text)

  def artikel_page(self, page):
    res = req.get(f'{self.url}/{page}')
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find_all('li', attrs={'data-cat':'#luxury'})
    return contents

  def get_berita(self):
    pta_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        respone = req.get(artikel.find('a', class_='gray button')['href'])
        soup_data = BeautifulSoup(respone.content, 'html.parser')
        cont = soup_data.find('div', attrs={'id':'content_journal'})
        # ambil data
        nama = cont.findAll('div')[1].div.span.text.split(' : ')[1]
        judul = str.join("", (cont.find('a', class_='title').string).splitlines()).capitalize()
        abstrak = str.join("", (cont.find('p').string).splitlines()).capitalize()
        pta_list.append({
            'Penulis' : nama,
            'Judul': judul,
            'Abstrak': abstrak
        })
    self.pta = pta_list
  
  def result(self):
    return pd.DataFrame(self.pta)
  
  def save(self, namefile='pta'):
    df = pd.DataFrame(self.pta).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [ ]:
scPta = ScrapingPtaTrunojoyo('https://pta.trunojoyo.ac.id/c_search/byfac/4')

In [ ]:
scPta.get_berita()

In [ ]:
scPta.result()

,Penulis,Judul,Abstrak
0,A.Ubaidillah S.Kom,Perancangan dan implementasi sistem database t...,Sistem informasi akademik (siakad) merupaka...
1,"M. Basith Ardianto,",Aplikasi kontrol dan monitoring jaringan kompu...,Berjalannya koneksi jaringan komputer dengan l...
2,"Akhmad Suyandi, S.Kom",Rancang bangun aplikasi proxy server untukenkr...,Web server adalah sebuah perangkat lunak serve...
3,Heri Supriyanto,Sistem pendukung keputusan optimasi penjadwala...,Penjadwalan kuliah di perguruan tinggi me...
4,Septian Rahman Hakim,Sistem augmented reality animasi benda bergera...,Seiring perkembangan teknologi yang ada diduni...
5,Adi Chandra Laksono,Gerak pekerja pada game real time strategy men...,Gerak pekerja ada pada game yang memiliki genr...
6,NURRACHMAT,Rancang bangun game perawatan sapi karapan men...,"Perkembangan game yang semakin pesat, memberik..."
7,Muhammad Choirur Rozi,Ekstraksi fitur berbasis two dimensional linea...,Sistem pengenalan wajah adalah suatu sistem un...
8,M Khoiril Anwar,Implementasi algoritma prim dan depth first ...,Teknologi mobile game beroperating system open...
9,Elky Lilik Misdayani,Sistem informasi perusahaan otobus sumber kenc...,Sistem informasi pengolahan data yang dibutuhk...


In [ ]:
scPta.save()

Berhasil di save


In [ ]:
class ScrapingKompas:
  def __init__(self, url):
    self.url = url

  def page(self):
    res_ = req.get(self.url)
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='paging__link')[:-2]
    return len(contents_)

  def artikel_page(self, page):
    res_ = req.get(f'{self.url}?page={page}')
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='article__link')
    return contents_

  def get_berita(self):
    berita_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        res = req.get(artikel['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        # ambil data
        judul = soup.find('h1', class_='read__title').string
        isi = isi = ' '.join(soup.find('div', class_='read__content').stripped_strings)
        berita_list.append({
            'Judul': judul,
            'Isi Berita': isi
        })
    self.beritas = berita_list
    return 
  
  def result(self):
    return pd.DataFrame(self.beritas)
  
  def save(self, namefile='beritaKompas'):
    df = pd.DataFrame(self.beritas).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [ ]:
data = ScrapingKompas('https://indeks.kompas.com/?site=nasional')

In [ ]:
data.get_berita()

In [ ]:
data.result()

,Judul,Isi Berita
0,"Hari Ini dalam Sejarah, Raja Faisal dari Arab ...",KOMPAS.com — Faisal bin Abdulaziz al-Saud dike...
1,"Tarif MRT Bundaran HI-Lebak Bulus Rp 8.500, LR...","JAKARTA, KOMPAS.com - Dalam rapat pimpinan gab..."
2,"MUI Minta Masyarakat Stop Pakai Istilah ""Cebon...","Jakarta, KOMPAS.com — Majelis Ulama Indonesia ..."
3,Pemantau Asing dari 33 Negara Akan Ikut Memoni...,"JAKARTA, KOMPAS.com — Komisioner Komisi Pemili..."
4,Ridwan Kamil Optimistis Tol Cisumdawu Bawa Per...,"BANDUNG , KOMPAS.com - Perkembangan pengerjaan..."
5,5 Fakta Turis Asing Ingin Dekati Kawah hingga ...,KOMPAS.com - Seorang wisatawan asing di kawasa...
6,TKN Jokowi-Ma'ruf Sebut Ada Bendera HTI di Kam...,"JAKARTA, KOMPAS.com - Anggota Dewan Pengarah B..."
7,Cemburu Jadi Motif Pelaku Bunuh Pria yang Jena...,"BATAM , KOMPAS.com - Kasus pembunuhan pria yan..."
8,Video: Detik-detik Jatuhnya Meteor 10 Kali Bom...,"KOMPAS.com - 18 Desember tahun lalu, sebuah me..."
9,Survei Charta Politika: Elektabilitas PSI Tert...,JAKARTA KOMPAS.com - Survei terbaru Charta Pol...


In [ ]:
data.save()

In [18]:
class ScrapingDetikcom:
  def __init__(self, url):
    self.url = url
  
  def page(self):
    res_ = req.get(self.url)
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.find('div', class_='pagination').findChildren()
    return int(contents_[-2].text)

  def artikel_page(self, page):
    res_ = req.get(f'{self.url}/{page}')
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='media__link')
    return contents_
  
  def get_berita(self):
    berita_list = []
    page = 5  # self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in range(1, len(artikels), 2):
        res = req.get(artikels[artikel]['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        # hapus tag style script
        for d in soup(['style', 'script']):
          d.decompose()
        # ambil data
        judul = (soup.find('h1', class_='detail__title'))
        isi = (soup.find('div', class_='detail__body-text'))
        if judul == None and isi == None:
          continue
        berita_list.append({
            'Judul': ' '.join(judul.stripped_strings),
            'Isi Berita': ' '.join(isi.stripped_strings)
        })
    self.beritas = berita_list

  def result(self):
    return pd.DataFrame(self.beritas)
  
  def save(self, namefile='beritaDetik'):
    df = pd.DataFrame(self.beritas).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [19]:
detikcom = ScrapingDetikcom('https://news.detik.com/pemilu')

In [20]:
detikcom.get_berita()

ConnectionError: ignored

In [ ]:
detikcom.result()